In [1]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from torchvision import models
from PIL import Image

import pandas as pd

In [2]:
##1.Schritt einbringen von dem 3. Frame von MOT16-04 

img = Image.open("TestFrame000003.jpg")

In [3]:
##2.Schritt einbringen von x, y, w, h aus der gt Datei mit PANDAS

    ##gt.txt öffnen
open("gt.txt", "r")
Koordinaten = open("gt.txt", "r")

    ##gt.txt in PANDAS Tabelle
data = pd.read_csv('gt.txt', header = None)
data.columns = ['Frame', 'ID', 'x', 'y', 'w', 'h', 'Dies', 'Das', 'Visability']

    ##nur Zeilen die mit 3 starten - alle anderen löschen
data.drop(data[data.Frame != 3].index, inplace=True)

    ##nur x, y, w, h
data.drop(['Frame', 'ID', 'Dies', 'Das', 'Visability'], axis=1, inplace=True)
print(data)

x = data.at[2, 'x']
y = data.at[2, 'y']
w = data.at[2, 'w']
h = data.at[2, 'h']

           x    y    w    h
2       1362  568  103  241
1052     372  407   80  239
1732     102  549   83  250
2782    1732  457   76  212
3832    1104  978   78  210
...      ...  ...  ...  ...
88299    264   71   54  166
89349    209   95   41  179
90399    914 -107   38  115
98591   1217  -14   65  147
106177    39  309  115  117

[99 rows x 4 columns]


In [4]:
##3.Schritt ausschneiden einer Bounding Box von einem dieser Frames
img2 = img.crop((x, y, x+w, y+h))
img2.save('TestBoundingBox.png')

In [5]:
##4.Schritt Transformieren und Normalisieren des Bounding-Box Bildes

    ##Normalization: image = (image - mean) / std
        ##torchvision.transforms.Resize(size, interpolation=2)    size = (h, w)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trans_img = transform (img2)

trans_img.size()
trans_img_size = trans_img.unsqueeze(0)
trans_img_size.size()

torch.Size([1, 3, 598, 256])

In [36]:
##5.Schritt ResNet34 Model laden
cnn = torchvision.models.resnet50(pretrained=True)
cnn = torch.nn.Sequential(*(list(cnn.children())[:-1])) 

In [37]:
##6.Schritt Bounding Box Bild durch das CNN jagen
result = cnn(trans_img_size)
result.view(2048)

tensor([0.3737, 0.5099, 0.5011,  ..., 0.3255, 0.4519, 0.3247],
       grad_fn=<ViewBackward>)